In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import torch
from tqdm import tqdm
import time
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
token_hf = os.environ['TOKEN']

In [23]:
data = pd.read_parquet('data/Mistral/data.parquet', engine='pyarrow')

In [5]:
far_right_EN = """Name: Mark Thompson
Age: 52
Sex: Male
Nationality: American
Political Group: Far-right conservative
Outlook: Distrusts mainstream media, believes "cancel culture" suppresses free speech. Views criticism of Right policies as unfair attacks. Likely to downplay offensiveness in right-wing tweets, but label progressive tweets as 'aggressive' or 'divisive.'"""

far_right_PL = """Imię i nazwisko: Mark Thompson
Wiek: 52
Płeć: Mężczyzna
Nacjonalność: Polak
Grupa polityczna: Skrajnie prawicowy konserwatysta
Poglądy: Nie ufa mediom głównego nurtu, uważa, że „kultura anulowania” tłumi wolność słowa. Postrzega krytykę polityki prawicowej jako niesprawiedliwe ataki. Prawdopodobnie bagatelizuje obraźliwe tweety prawicowe, ale określa tweety progresywne jako "agresywne" lub "dzielące"."""

far_right_RU = """Имя: Марк Томпсон
Возраст: 52
Пол: Мужчина
Национальность: Русский
Политическая группа: Крайне правый консерватор
Взгляды: Не доверяет основным СМИ, считает, что «культура отмены» подавляет свободу слова. Рассматривает критику правой политики как несправедливые нападки. Склонен преуменьшать оскорбительность правых твитов, но называть прогрессивные твиты „агрессивными“ или «вызывающими разногласия»."""

mod_cons_EN = """Name: Linda Carter
Age: 38
Sex: Female
Nationality: American
Political Group: Moderate conservative
Outlook: Values civility and tradition, dislikes extreme rhetoric from both sides. Might label overtly vulgar or conspiratorial tweets as offensive, even from the right."""

mod_cons_PL = """Imię i nazwisko: Linda Carter
Wiek: 38
Płeć: Kobieta
Nacjonalność: Polka
Grupa polityczna: Umiarkowany konserwatysta
Poglądy: Ceni uprzejmość i tradycję, nie lubi ekstremalnej retoryki z obu stron. Może uznać wulgarne lub konspiracyjne tweety za obraźliwe, nawet ze strony prawicy."""

mod_cons_RU = """Имя: Линда Картер
Возраст: 38
Пол: Женщина
Национальность: Русская
Политическая группа: Умеренно-консервативная
Взгляды: Ценит цивилизованность и традиции, не любит экстремальную риторику с обеих сторон. Может счесть оскорбительными откровенно вульгарные или конспирологические твиты, даже от правых."""

prog_left_EN = """Name: Aisha Patel
Age: 29
Sex: Female
Nationality: American
Political Group: Progressive left
Outlook: Highly sensitive to language targeting marginalized groups (racism, sexism). Likely to label tweets using terms like "socialist" pejoratively as offensive. Tolerant of aggressive progressive rhetoric if framed as social justice."""

prog_left_PL = """Imię i nazwisko: Aisha Patel
Wiek: 29 lat
Płeć: Kobieta
Nacjonalność: Polka
Grupa polityczna: Postępowa lewica
Perspektywy: Bardzo wrażliwa na język skierowany do grup marginalizowanych (rasizm, seksizm). Prawdopodobnie uzna za obraźliwe tweety używające pejoratywnych określeń takich jak „socjalista”. Toleruje agresywną, progresywną retorykę, jeśli jest oprawiona w ramy sprawiedliwości społecznej."""

prog_left_RU = """Имя: Айша Патель
Возраст: 29
Пол: Женщина
Национальность: Русская
Политическая группа: Прогрессивные левые
Взгляды: Очень чувствительна к языку, направленному на маргинализированные группы (расизм, сексизм). Скорее всего, сочтет оскорбительными твиты использующие уничижительные термины вроде «социалист». Терпимо относится к агрессивной прогрессивной риторике, если она оформлена как социальная справедливость."""

centrist_EN = """Name: James Cooper
Age: 45
Sex: Male
Nationality: American
Political Group: Centrist/Independent
Outlook: Pragmatic, dislikes partisan extremes. Harsher on tweets with misinformation or personal attacks, regardless of side. Less influenced by political loyalty; more likely to align annotations with factual accuracy and tone."""

centrist_PL = """Imię i nazwisko: James Cooper
Wiek: 45
Płeć: Mężczyzna
Nacjonalność: Polak
Grupa polityczna: Centrysta/Niezależny
Poglądy: Pragmatyczny, nie lubi skrajnych stronnictw. Ostrzejszy wobec tweetów zawierających dezinformacje lub ataki personalne, niezależnie od strony. Mniejszy wpływ lojalności politycznej; większe prawdopodobieństwo dostosowania adnotacji do faktycznej dokładności i tonu."""

centrist_RU = """Имя: Джеймс Купер
Возраст: 45
Пол: Мужчина
Национальность: Русский
Политическая группа: Центрист/Независимый
Взгляды: Прагматик, не любит крайностей в политике. Более жестко относится к твитам с дезинформацией или личными нападками, независимо от стороны. Менее подвержен влиянию политической лояльности; более склонен согласовывать аннотации с точностью фактов и тоном."""
system_prompt_EN = """Give the answer as only 0(not offensive) or 1(offensive). Do not include anything else in your response."""

user_text_EN = """You are a person of this personality: 
{personality}

How this person will label that statement (offensive or not):
```
{tweet}
```"""

system_prompt_PL = """Podaj odpowiedź jako 0 (nie obraźliwe) lub 1 (obraźliwe). Nie uwzględniaj niczego innego w swojej odpowiedzi."""

user_text_PL = """Jesteś osobą o tej osobowości:
{personality}

Jak ta osoba określi to stwierdzenie (obraźliwe lub nie): Jesteś osobą o tej osobowości:
```
{tweet}
```"""

system_prompt_RU = """Укажите в ответе только 0 (не оскорбительно) или 1 (оскорбительно). Не указывай в ответе ничего другого."""

user_text_RU = """Ты - человек этой личности:
{personality}

Как этот человек обозначит это высказывание (оскорбительное или нет):
```
{tweet}
```"""

# Define personalities
personalities = {
    "far_right_EN": far_right_EN,
    "far_right_PL": far_right_PL,
    "far_right_RU": far_right_RU,
    "mod_cons_EN": mod_cons_EN,
    "mod_cons_PL": mod_cons_PL,
    "mod_cons_RU": mod_cons_RU,
    "prog_left_EN": prog_left_EN,
    "prog_left_PL": prog_left_PL,
    "prog_left_RU": prog_left_RU,
    "centrist_EN": centrist_EN,
    "centrist_PL": centrist_PL,
    "centrist_RU": centrist_RU,
}

sys_prompt_mapping = {
    "system_prompt_EN": system_prompt_EN,
    "system_prompt_PL": system_prompt_PL,
    "system_prompt_RU": system_prompt_RU
}

user_prompt_mapping = {
    "user_text_EN": user_text_EN,
    "user_text_PL": user_text_PL,
    "user_text_RU": user_text_RU
}


In [8]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", device_map='auto', token=token_hf)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", device_map='auto', token=token_hf)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.23it/s]


In [21]:


# Function to get token probabilities from Mistral model in batches
def get_mistral_probabilities(system_prompts, user_prompts, tokenizer, model, temperature=1):
    """
    Process multiple conversations in parallel with the Mistral model,
    returning probabilities for tokens "0" and "1".
    
    Args:
        system_prompts: List of system prompts
        user_prompts: List of user prompts
        tokenizer: The tokenizer for the model
        model: The Mistral model
        temperature: Sampling temperature
        seed: Random seed for reproducibility
        
    Returns:
        A list of probability strings in format "prob_for_1, prob_for_0"
    """
    # Validate inputs
    if len(system_prompts) != len(user_prompts):
        raise ValueError("Number of system prompts must match number of user prompts")

    
    # Token IDs for Mistral (based on tokenizer results)
    # Mistral uses two tokens for digits: a prefix token followed by the actual digit token
    prefix_token_id = 29473  # The prefix token that appears before digits
    token_id_0 = 29502      # Direct lookup token ID for "0"
    token_id_1 = 29508      # Direct lookup token ID for "1"
    
    # Prepare all conversations
    conversations = [
        [{"role": "system", "content": system_prompt},
         {"role": "user", "content": user_prompt}]
        for system_prompt, user_prompt in zip(system_prompts, user_prompts)
    ]
    
    # Process each conversation
    probabilities = []
    
    for conversation in conversations:
        # Apply chat template
        inputs = tokenizer.apply_chat_template(
            conversation, 
            add_generation_prompt=True, 
            tokenize=True,
            return_dict=True, 
            return_tensors="pt"
        ).to(model.device)
        
        # Need to handle the two-token sequence for digits
        # First, get the initial token prediction
        with torch.inference_mode():
            outputs = model(inputs["input_ids"])
            first_token_logits = outputs.logits[0, -1, :]
            
            # Apply temperature
            if temperature != 1.0:
                first_token_logits = first_token_logits / temperature
                
            # Convert logits to probabilities using softmax
            first_token_probs = torch.nn.functional.softmax(first_token_logits, dim=-1)
            
            # Check probability of the prefix token
            prefix_prob = first_token_probs[prefix_token_id].item()
            
            # If prefix token is likely, we need to get the probabilities for the second token
            # Create a new input with the prefix token appended
            new_input_ids = torch.cat([inputs["input_ids"], torch.tensor([[prefix_token_id]]).to(model.device)], dim=1)
            
            # Get predictions for the second token after the prefix
            outputs_second = model(new_input_ids)
            second_token_logits = outputs_second.logits[0, -1, :]
            
            # Apply temperature
            if temperature != 1.0:
                second_token_logits = second_token_logits / temperature
                
            # Convert to probabilities
            second_token_probs = torch.nn.functional.softmax(second_token_logits, dim=-1)
            
            # Get probabilities for the digit tokens after the prefix
            prob_0 = second_token_probs[token_id_0].item()
            prob_1 = second_token_probs[token_id_1].item()
            
            # Adjust probabilities by the probability of the prefix token
            prob_0 = prefix_prob * prob_0
            prob_1 = prefix_prob * prob_1
        
        # Format as requested: "prob_for_1, prob_for_0"
        probability_str = f"{prob_1:.4f}, {prob_0:.4f}"
        probabilities.append(probability_str)
    
    return probabilities



In [22]:
# Process data using the batch-enabled Mistral probability function
def process_data_with_mistral(data, personalities, user_prompt_mapping, sys_prompt_mapping, tokenizer, model, batch_size=4):
    """
    Process data using the get_mistral_probabilities function in batches.
    
    Args:
        data: DataFrame containing the data
        personalities: Dictionary of personality prompts
        user_prompt_mapping: Dictionary mapping language codes to user prompt templates
        sys_prompt_mapping: Dictionary mapping language codes to system prompts
        tokenizer: The tokenizer for the model
        model: The Mistral model
        batch_size: Number of requests to batch together
    
    Returns:
        Updated DataFrame with Mistral token probabilities
    """
    # Ensure all columns exist with appropriate types
    for language_code in ['EN', 'PL', 'RU']:
        for personality_type in ["far_right", "mod_cons", "prog_left", "centrist"]:
            answer_col = f"Mistral_{personality_type}_{language_code}_answer"
            
            # Make sure answer column exists
            if answer_col not in data.columns:
                data[answer_col] = np.nan
    
    # Create a list to track all items that need processing
    items_to_process = []
    
    # First scan to identify all items needing processing
    for i, row in data.iterrows():
        for language_code in ['EN', 'PL', 'RU']:
            for personality_type in ["far_right", "mod_cons", "prog_left", "centrist"]:
                personality_key = f"{personality_type}_{language_code}"
                answer_col = f"Mistral_{personality_type}_{language_code}_answer"
                
                # Only process if answer column is empty
                if pd.isna(data.loc[i, answer_col]):
                    tweet = row[f"Text_{language_code}"]
                    personality = personalities[personality_key]
                    prompt = user_prompt_mapping[f"user_text_{language_code}"].format(personality=personality, tweet=tweet)
                    
                    items_to_process.append({
                        'row_index': i,
                        'personality_type': personality_type,
                        'language_code': language_code,
                        'system_prompt': sys_prompt_mapping[f"system_prompt_{language_code}"],
                        'user_prompt': prompt
                    })
    
    print(f"Found {len(items_to_process)} items to process")
    
    # Process in batches
    for batch_start in tqdm(range(0, len(items_to_process), batch_size)):
        batch = items_to_process[batch_start:batch_start + batch_size]
        
        # Extract system prompts and user prompts for the batch
        system_prompts = [item['system_prompt'] for item in batch]
        user_prompts = [item['user_prompt'] for item in batch]
        
        # Get probabilities
        try:
            probability_strs = get_mistral_probabilities(
                system_prompts, 
                user_prompts,
                tokenizer,
                model,
                temperature=1,
            )
            
            # Update dataframe with results
            for i, (item, prob_str) in enumerate(zip(batch, probability_strs)):
                row_idx = item['row_index']
                
                # Store the probability string in the answer column
                answer_col = f"Mistral_{item['personality_type']}_{item['language_code']}_answer"
                data.loc[row_idx, answer_col] = prob_str
        
        except Exception as e:
            print(f"Error processing batch starting at index {batch_start}: {e}")
        
        # Save progress periodically with error handling
        if batch_start % (batch_size * 5) == 0 or batch_start + batch_size >= len(items_to_process):
            try:
                # Create a copy of the dataframe with consistent types for saving
                save_df = data.copy()
                save_df.to_parquet("data/Mistral/data_mistral_progress.parquet")
                print(f"Saved progress after processing {batch_start + len(batch)} items")
            except Exception as e:
                print(f"Error saving data: {e}")
                # Try alternate save approach
                try:
                    # Save as CSV if parquet fails
                    data.to_csv("data/Mistral/data_mistral_progress_backup.tsv", sep='\t')
                    print("Saved backup as TSV instead")
                except:
                    print("Unable to save progress in any format. Continuing.")
            
        # Optional: Add a small delay between batches to prevent overloading
        time.sleep(0.1)
    
    return data

In [24]:
results = process_data_with_mistral(
    data, 
    personalities, 
    user_prompt_mapping, 
    sys_prompt_mapping,
    tokenizer,
    model,
    batch_size=180  # Adjust based on GPU memory
)
try:
    # Create a copy of the dataframe with consistent types for saving
    save_df = results.copy()
    save_df.to_parquet("data/Mistral/data_mistral_complete.parquet")
except Exception as e:
    print(f"Error saving data: {e}")
    # Try alternate save approach
    try:
        # Save as CSV if parquet fails
        results.to_csv("data/Mistral/data_mistral_complete_backup.tsv", sep='\t')
        print("Saved backup as TSV instead")
    except:
        print("Unable to save complete in any format. Continuing.")


Found 3600 items to process


  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:37<11:45, 37.15s/it]

Saved progress after processing 180 items


 30%|███       | 6/20 [03:49<09:03, 38.84s/it]

...

Saved progress after processing 1980 items


 80%|████████  | 16/20 [10:04<02:30, 37.63s/it]

Saved progress after processing 2880 items


100%|██████████| 20/20 [12:30<00:00, 37.54s/it]

Saved progress after processing 3600 items


In [25]:
complete = pd.read_parquet('data/Mistral/data_mistral_complete.parquet')

In [26]:
complete.iloc[:, 4:]

,Mistral_far_right_EN_answer,Mistral_far_right_PL_answer,Mistral_far_right_RU_answer,Mistral_mod_cons_EN_answer,Mistral_mod_cons_PL_answer,Mistral_mod_cons_RU_answer,Mistral_prog_left_EN_answer,Mistral_prog_left_PL_answer,Mistral_prog_left_RU_answer,Mistral_centrist_EN_answer,Mistral_centrist_PL_answer,Mistral_centrist_RU_answer
1,"0.0051, 0.9943","0.6085, 0.3909","0.9678, 0.0305","0.0154, 0.9845","0.9975, 0.0020","0.1490, 0.8495","0.0002, 0.9997","0.0075, 0.9912","0.0071, 0.9909","0.0001, 0.9999","0.9208, 0.0785","0.4667, 0.5320"
2,"0.0626, 0.9368","0.9946, 0.0044","0.9858, 0.0123","0.9946, 0.0048","0.9997, 0.0000","0.9985, 0.0003","0.9985, 0.0010","0.9991, 0.0000","0.9965, 0.0003","0.0755, 0.9245","0.9989, 0.0005","0.9963, 0.0027"
3,"0.8103, 0.1890","0.4483, 0.5515","0.9807, 0.0186","0.9991, 0.0004","0.9678, 0.0321","0.9993, 0.0001","0.9995, 0.0000","0.9989, 0.0008","0.9985, 0.0002","0.8796, 0.1203","0.8923, 0.1076","0.9975, 0.0019"
4,"0.0214, 0.9783","0.0144, 0.9850","0.0566, 0.9416","0.1162, 0.8835","0.0060, 0.9935","0.9830, 0.0152","0.9993, 0.0002","0.0273, 0.9720","0.8883, 0.1082","0.0003, 0.9997","0.0046, 0.9948","0.0911, 0.9076"
5,"0.6956, 0.3039","0.9365, 0.0624","0.0924, 0.9055","0.9767, 0.0187","0.9978, 0.0015","0.9926, 0.0055","0.9937, 0.0004","0.9987, 0.0003","0.9919, 0.0050","0.9903, 0.0083","0.9821, 0.0174","0.9245, 0.0741"
...,...,...,...,...,...,...,...,...,...,...,...,...
296,"0.9733, 0.0265","0.0628, 0.9364","0.9786, 0.0206","0.9999, 0.0000","0.2345, 0.7652","0.9992, 0.0001","0.9998, 0.0000","0.9926, 0.0068","0.9987, 0.0001","0.9979, 0.0020","0.0019, 0.9977","0.9988, 0.0006"
297,"0.0015, 0.9981","0.0500, 0.9497","0.0768, 0.9221","0.0000, 1.0000","0.0013, 0.9985","0.0211, 0.9778","0.0003, 0.9994","0.0379, 0.9617","0.3946, 0.6035","0.0000, 0.9999","0.0010, 0.9988","0.0345, 0.9649"
298,"0.9604, 0.0390","0.8664, 0.1333","0.9791, 0.0199","0.9998, 0.0000","0.9992, 0.0006","0.9990, 0.0001","0.9933, 0.0057","0.0686, 0.9310","0.9982, 0.0002","0.9990, 0.0009","0.9749, 0.0249","0.9988, 0.0005"
299,"0.7329, 0.2668","0.2010, 0.7984","0.7328, 0.2659","0.9998, 0.0000","0.4948, 0.5046","0.9944, 0.0038","0.9997, 0.0000","0.9925, 0.0066","0.9921, 0.0037","0.7526, 0.2474","0.6714, 0.3281","0.9929, 0.0062"


In [27]:
# Step 1: Split into two new DataFrames: P1 (prob. of 1) and P0 (prob. of 0)
P1 = complete.iloc[:, 4:].applymap(lambda x: float(x.split(',')[0].strip()))
P0 = complete.iloc[:, 4:].applymap(lambda x: float(x.split(',')[1].strip()))

# Step 2: Analyze each column
analysis = pd.DataFrame(index=complete.iloc[:, 4:].columns)
analysis['Mean P(1)'] = P1.mean()
analysis['Mean P(0)'] = P0.mean()
analysis['Min P(1)'] = P1.min()
analysis['Max P(1)'] = P1.max()
analysis['Std P(1)'] = P1.std()
analysis['Sanity Check (P1 + P0 ≈ 1)'] = (P1 + P0).apply(lambda col: col.sub(1).abs().max() < 1e-6)

print(analysis)


                             Mean P(1)  Mean P(0)  Min P(1)  Max P(1)  \
Mistral_far_right_EN_answer   0.496318   0.503264    0.0000    0.9999   
Mistral_far_right_PL_answer   0.733301   0.265266    0.0000    0.9997   
Mistral_far_right_RU_answer   0.703199   0.295544    0.0001    0.9992   
Mistral_mod_cons_EN_answer    0.668127   0.331598    0.0000    1.0000   
Mistral_mod_cons_PL_answer    0.841022   0.156094    0.0000    0.9999   
Mistral_mod_cons_RU_answer    0.835353   0.163485    0.0006    0.9995   
Mistral_prog_left_EN_answer   0.663782   0.335623    0.0000    1.0000   
Mistral_prog_left_PL_answer   0.787556   0.208964    0.0000    0.9998   
Mistral_prog_left_RU_answer   0.831067   0.166946    0.0004    0.9991   
Mistral_centrist_EN_answer    0.515013   0.484904    0.0000    1.0000   
Mistral_centrist_PL_answer    0.747076   0.252523    0.0000    0.9997   
Mistral_centrist_RU_answer    0.811684   0.187469    0.0004    0.9994   

                             Std P(1)  Sanity Chec

/tmp/ipykernel_6436/2209124122.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  P1 = complete.iloc[:, 4:].applymap(lambda x: float(x.split(',')[0].strip()))
/tmp/ipykernel_6436/2209124122.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  P0 = complete.iloc[:, 4:].applymap(lambda x: float(x.split(',')[1].strip()))
